In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_name(1))

True
2
NVIDIA RTX 6000 Ada Generation
NVIDIA RTX 6000 Ada Generation


In [ ]:
http://dandelion-ollama-1:11434

In [7]:
!curl http://dandelion-ollama-1:11434/api/embeddings | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    22  100    22    0     0  12134      0 --:--:-- --:--:-- --:--:-- 22000
405
parse error: Invalid numeric literal at line 1, column 11


In [14]:
# list models
!curl http://dandelion-ollama-1:11434/api/tags | jq | grep embed

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2052    0  2052    0     0   816k      0 --:--:-- --:--:-- --:--:-- 1001k
      "name": "snowflake-arctic-embed:137m",
      "model": "snowflake-arctic-embed:137m",
      "name": "snowflake-arctic-embed:335m",
      "model": "snowflake-arctic-embed:335m",
      "name": "nomic-embed-text:latest",
      "model": "nomic-embed-text:latest",


In [ ]:
# pull model
curl -X POST http://dandelion-ollama-1:11434/api/pull -H "Content-Type: application/json" -d '{
  "name": "bge-m3:567m"
}'

In [4]:
import requests

# °²³] docker-compose ¸ÌªºªA°È¦WºÙ¬O ollama
response = requests.get("http://dandelion-ollama-1:11434/api/tags")
print(response.json())


{'models': [{'name': 'llama3.1:8b', 'model': 'llama3.1:8b', 'modified_at': '2025-07-16T10:00:52.352347186Z', 'size': 4920753328, 'digest': '46e0c10c039e019119339687c3c1757cc81b9da49709a3b3924863ba87ca666e', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'llama', 'families': ['llama'], 'parameter_size': '8.0B', 'quantization_level': 'Q4_K_M'}}, {'name': 'nomic-embed-text:latest', 'model': 'nomic-embed-text:latest', 'modified_at': '2025-07-16T09:48:21.743163953Z', 'size': 274302450, 'digest': '0a109f422b47e3a30ba2b10eca18548e944e8a23073ee3f3e947efcf3c45e59f', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'nomic-bert', 'families': ['nomic-bert'], 'parameter_size': '137M', 'quantization_level': 'F16'}}, {'name': 'qwen2.5:0.5b', 'model': 'qwen2.5:0.5b', 'modified_at': '2025-07-16T06:49:08.343900724Z', 'size': 397821319, 'digest': 'a8b0c51577010a279d933d14c2a8ab4b268079d44c5c8830c0a93900f1827c67', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'qwen2',


### pull model
`curl -X POST http://dandelion-ollama-1:11434/api/pull -H "Content-Type: application/json" -d '{ 
  "name": "bge-m3:567m" 
}' ` 

| 部份                                             | 說明                        |
| ------------------------------------------------ | -------------------------- |
| `curl -X POST`                                   | 用 POST 方法呼叫 HTTP API         |
| `http://dandelion-ollama-1:11434/api/pull`       | 呼叫 Ollama 服務器的 **pull 接口** |
| `-H "Content-Type: application/json"`            | 告訴伺服器這是 JSON 格式            |
| `-d '{ "name": "snowflake-arctic-embed:335m" }'` | 請求內容, 指定拉取的模型名稱         |


In [10]:
# https://python.langchain.com/docs/versions/v0_2/deprecations/
# 從 langchain 引入 pdf 載入器
# from langchain.document_loaders import PyPDFLoader #! 已棄用
from langchain_community.document_loaders import PyPDFLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter #! 已棄用
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

# 使用 ollama 的模型
# llm_model = OllamaLLM(
#     base_url='http://dandelion-ollama-1:11434', 
#     model="llama3.1:8b"
# )

# 使用 ollama 的 embedding 模型
embeddings_model = OllamaEmbeddings(
    base_url='http://dandelion-ollama-1:11434', 
    model="nomic-embed-text",
    # model="nomic-embed-text"
)

'''
nomic-embed-text
snowflake-arctic-embed:335m
'''


# ***********************
# 載入 PDF 文件
loader = PyPDFLoader('./data/PDF_file.pdf')
docs = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
documents = text_splitter.split_documents(docs)

# 將文檔寫入 ChromaDB
db = Chroma.from_documents(
    documents,
    embedding=embeddings_model,
    persist_directory="./nomic-db"
)


retriever = db.as_retriever()
query = "我想要一隻綿羊"
print("--------")
print("embeddings_model:", embeddings_model.model)
print("query:", query)
print("--------")

# k=3 表示返回最相似的3個文檔
# results = db.similarity_search(query,k = 3)
results = db.similarity_search(query)
print("相似度：", results)

print(f"找到 {len(results)} 個相關文檔\n")

c = 1;
for r in results:
    print(f"[{c}] {r.page_content}\n---\n")
    c = c +1


ValueError: File path ./data/PDF_file.pdf is not a valid file or url

In [13]:
!pip install -U matplotlib scikit-learn seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 699.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 857.0 kB/s eta 0:00:00 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 846.5 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 3.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 4.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 4.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 4.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 5.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 8.3 MB/s eta 0:00:0000:0100:01
